<a href="https://colab.research.google.com/github/graylan0/quantum-machine-learning/blob/main/Suno_BarkAI_TheBlokeLlama2_Emotional_Inference_Enabled_TTS_WORKING_WOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/suno-ai/bark.git

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-16r8u0at
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-16r8u0at
  Resolved https://github.com/suno-ai/bark.git to commit 61710e5265cbc2f5eb098e08c6bee5d23dfff3fd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 52.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 82.7 MB/s eta 0:00:00
     ━

In [ ]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822309 sha256=4b8779163eed69216fcfc78d4ef5f40010528cf299662d6c372c4e85ad9b87d4
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [ ]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin

--2023-09-23 10:25:40--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.124, 18.172.134.24, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/30/e3/30e3aca7233f7337633262ff6d59dd98559ecd8982e7419b39752c8d0daae1ca/3bfdde943555c78294626a6ccd40184162d066d39774bd2c98dae24943d32cc3?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b-chat.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1695723940&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NTcyMzk0MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zMC9lMy8zMGUzYWNhNzIzM2Y3MzM3NjMzMjYyZmY2ZDU5ZGQ5ODU1OWVjZDg5ODJlNzQxOWIzOTc1MmM4

In [13]:
import re
import llama_cpp
from bark import SAMPLE_RATE, generate_audio
import numpy as np
from scipy.io.wavfile import write as write_wav
import uuid

def llama2_generate_text(prompt):
    llm = llama_cpp.Llama(
        model_path="llama-2-7b-chat.ggmlv3.q8_0.bin",
        n_gpu_layers=-1,
        n_ctx=3900,
    )
    response = llm(prompt, max_tokens=800)
    del llm
    return response['choices'][0]['text']

def extract_sentences_and_emotions(text):
    pattern = r'[\[\(](\w+)[\]\)] ([^\.]+\.?)'
    return re.findall(pattern, text)

def generate_response(message):
    prompt = f"""
    1. Analyze the following text: '{message}' and add emotional taging into the text segements by spliting the text up into segments for example; [happy] [sad] [neutral]
    2. Split the text into meaningful segments.
    3. Include emotional tags in the responses.
    """
    segments_analysis = llama2_generate_text(prompt)
    sentences_and_emotions = extract_sentences_and_emotions(segments_analysis)

    if not sentences_and_emotions:
        print("No sentences and emotions found.")
        return

    pieces = []
    for emotion, sentence in sentences_and_emotions:
        if emotion == "happy":
            emotion_tag = "[joyful]"
        elif emotion == "sad":
            emotion_tag = "[sad]"
        elif emotion == "angry":
            emotion_tag = "[angry]"
        elif emotion == "calm":
            emotion_tag = "[calm]"
        elif emotion == "excited":
            emotion_tag = "[excited]"
        else:
            emotion_tag = "[neutral]"

        audio_array = generate_audio(sentence + emotion_tag, history_prompt="v2/en_speaker_6")
        silence = np.zeros(int(0.75 * SAMPLE_RATE))
        pieces += [audio_array, silence.copy()]

    audio = np.concatenate(pieces)
    wav_file_name = str(uuid.uuid4()) + ".wav"
    write_wav(wav_file_name, SAMPLE_RATE, audio)
    print(f"Audio file generated: {wav_file_name}")

# Get input text from the user at runtime
user_input = input("Enter your text: ")

generate_response(user_input)


Enter your text: Taco Taco. Build better tacos. Love your enemy.


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 134MB/s]
100%|██████████| 28/28 [00:27<00:00,  1.01it/s]


Audio file generated: ee543a32-1917-4e3e-a042-4060678ad49b.wav
